# Setup

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
tf.__version__

'2.2.0'

In [4]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [5]:
items = pd.read_csv("../Datasets/ml-100k/Text/items.csv")

In [6]:
data_text = items['Summary']

In [7]:
data_text[0]

'A little boy named Andy loves to be in his room, playing with his toys, especially his doll named "Woody". But, what do the toys do when Andy is not with them, they come to life. Woody believes that his life (as a toy) is good. However, he must worry about Andy\'s family moving, and what Woody does not know is about Andy\'s birthday party. Woody does not realize that Andy\'s mother gave him an action figure known as Buzz Lightyear, who does not believe that he is a toy, and quickly becomes Andy\'s new favorite toy. Woody, who is now consumed with jealousy, tries to get rid of Buzz. Then, both Woody and Buzz are now lost. They must find a way to get back to Andy before he moves without them, but they will have to pass through a ruthless toy killer, Sid Phillips. —John Wiggins'

# AutoEncoder
## Text Data

In [8]:
import re

from keras.optimizers import Adam
from keras.models import Model, Sequential

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import wordnet
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer 
from nltk.tokenize import word_tokenize as wt

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.losses import sparse_categorical_crossentropy
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding, Bidirectional

Using TensorFlow backend.
[nltk_data] Downloading package punkt to /home/sriram/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sriram/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sriram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
lem = wordnet.WordNetLemmatizer()
sw = set(stopwords.words("english")) 

In [12]:
class Autoencoder_Text(Model):
    def __init__(self, input_=300, hidden=50, vocab=13672, train=True, data_text):
        super(Autoencoder_Text, self).__init__()
        self.max_len = input_
        self.hidden = hidden
        self.vocab = vocab
        
        self.pad_sentence = self.pre_process(data_text, True)
        emb_matrix, embedding_vector, embed_vector_len = self.read_glove_vector('./glove.6B.100d.txt')
            
        self.encoder = Sequential([
            Embedding(input_dim = self.vocab, output_dim=embed_vector_len, 
                      input_length=self.max_len, weights=[emb_matrix], trainable=False),
            Bidirectional(LSTM(self.hidden, return_sequences=False))
        ])
        
        self.r_vec = RepeatVector(self.max_len)
        self.decoder = Sequential([
            Bidirectional(LSTM(self.hidden, return_sequences=True, dropout=0.2)),
            TimeDistributed(Dense(self.vocab))
        ])
        self.enc_dec_model = Sequential([self.encoder, self.r_vec, self.decoder])
    
    def clean_text(self, text):
        text = str(text).lower()
        text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
        text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
        text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
        text = re.sub('\t', ' ',  text)
        text = re.sub(r" +", ' ', text)
        text = wt(text)
        text = (" ").join([lem.lemmatize(i, pos ='v') 
                           for i in text if i not in sw])
        return(text)
    
    def tokenize(self, sentences):
        text_tokenizer = Tokenizer()
        text_tokenizer.fit_on_texts(sentences)
        return(text_tokenizer.texts_to_sequences(sentences), text_tokenizer)
    
    def pre_process(self, data_text, train=False):
        data_text = data_text.apply(self.clean_text)
        text_tokenized, self.text_tokenizer = self.tokenize(data_text)
        
        if train:
            self.vocab = len(self.text_tokenizer.word_index) + 1
            self.max_len = int(len(max(text_tokenized,key=len)))
            
        pad_sentence = pad_sequences(text_tokenized, self.max_len, padding = "post")
        return(pad_sentence.reshape(*pad_sentence.shape, 1))
        
    def read_glove_vector(self, glove_vec):
        with open(glove_vec, 'r', encoding='UTF-8') as f:
            words = set()
            word_to_vec_map = {}
            for line in f:
                w_line = line.split()
                curr_word = w_line[0]
                word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)
                
        embed_vector_len = word_to_vec_map['moon'].shape[0]
        emb_matrix = np.zeros((self.vocab, embed_vector_len))

        for word, index in self.text_tokenizer.word_index.items():
            embedding_vector = word_to_vec_map.get(word)
            if embedding_vector is not None:
                emb_matrix[index, :] = embedding_vector
        return(emb_matrix, embedding_vector, embed_vector_len)
    
#     def model(self):
#         emb_matrix, embedding_vector, embed_vector_len = self.read_glove_vector('./glove.6B.100d.txt')
#         self.encoder = Sequential([
#             Embedding(input_dim = self.vocab, output_dim=embed_vector_len, 
#                       input_length=self.max_len, weights=[emb_matrix], trainable=False),
#             Bidirectional(LSTM(self.hidden, return_sequences=False))
#         ])
        
#         self.r_vec = RepeatVector(self.max_len)
#         self.decoder = Sequential([
#             Bidirectional(LSTM(self.hidden, return_sequences=True, dropout=0.2)),
#             TimeDistributed(Dense(self.vocab))
#         ])
#         self.enc_dec_model = Sequential([self.encoder, self.r_vec, self.decoder])
    
    def train(self, data_text, epochs=10, batch_size=20):
        # pad_sentence = self.pre_process(data_text, True)
        # self.model()
        self.__init__()
        self.enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
        
        self.enc_dec_model.summary()
        self.enc_dec_model.fit(np.squeeze(self.pad_sentence, axis = 2), 
                                      self.pad_sentence, batch_size=batch_size, epochs=epochs)
        self.enc_dec_model.save('./pretrained/text_model')
        
    def call(self, inputs):
        pad_sentences = self.pre_process(inputs, False)
        print(tf.convert_to_tensor(pad_sentences).shape)
        return(self.enc_dec_model(tf.squeeze(pad_sentences, axis = 2)))

In [ ]:
AE = Autoencoder_Text(data_text)
AE.train(data_text, epochs = 10, batch_size = 30)

100 (13672, 100)
100 (13672, 100)
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_10 (Sequential)   (None, 100)               1427600   
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 240, 100)          0         
_________________________________________________________________
sequential_11 (Sequential)   (None, 240, 13672)        1441272   
Total params: 2,868,872
Trainable params: 1,501,672
Non-trainable params: 1,367,200
_________________________________________________________________
Epoch 1/10
1682/1682 [==============================] - 65s 39ms/step - loss: 3.8446 - accuracy: 0.7539
Epoch 2/10
1682/1682 [==============================] - 64s 38ms/step - loss: 3.0107 - accuracy: 0.7852
Epoch 3/10
1682/1682 [==============================] - 56s 33ms/step - loss: 3.0422 - accuracy: 0.7758
Epoch 4/10
1682/168

In [ ]:
inputs = np.squeeze(AE.pre_process(data_text), axis = 2)
output = AE.encoder.predict(inputs)
print(output[:2])